In [1]:
import numpy as np

import qiskit as qk

import tools as t

from qiskit.circuit.library import MCMT

from copy import deepcopy

import matplotlib.pyplot as plt

In [2]:
def histo(dic, normalize=False):
    new_dic = {}
    total_shots = np.sum(t.bit_array(dic))

    for key, value in dic.items():
        new_dic[key] = value / total_shots
    print(dic)
    h = t.plot_weights(new_dic)
    plt.show()
    return

In [3]:
# build W|0> = |who> operator

qr = qk.QuantumRegister(6, 'q')
qc = qk.QuantumCircuit(qr)

qc.h([0, 1, 3])
qc.cnot([0, 1, 3], [2, 5, 4])

print(qc.draw())

W_gate = deepcopy(qc)

     ┌───┐          
q_0: ┤ H ├──■───────
     ├───┤  │       
q_1: ┤ H ├──┼────■──
     └───┘┌─┴─┐  │  
q_2: ─────┤ X ├──┼──
     ┌───┐└───┘  │  
q_3: ┤ H ├──■────┼──
     └───┘┌─┴─┐  │  
q_4: ─────┤ X ├──┼──
          └───┘┌─┴─┐
q_5: ──────────┤ X ├
               └───┘


In [4]:
# build W_dag gate
qr = qk.QuantumRegister(6, 'q')
qc = qk.QuantumCircuit(qr)

qc.cnot([0, 1, 3], [2, 5, 4])
qc.h([0, 1, 3])

print(qc.draw())

Wdag_gate = deepcopy(qc)

          ┌───┐     
q_0: ──■──┤ H ├─────
       │  └───┘┌───┐
q_1: ──┼────■──┤ H ├
     ┌─┴─┐  │  └───┘
q_2: ┤ X ├──┼───────
     └───┘  │  ┌───┐
q_3: ──■────┼──┤ H ├
     ┌─┴─┐  │  └───┘
q_4: ┤ X ├──┼───────
     └───┘┌─┴─┐     
q_5: ─────┤ X ├─────
          └───┘     


In [5]:
# show that |who> is correct
qr = qk.QuantumRegister(6, 'q')
qc = qk.QuantumCircuit(qr)

qc.compose(W_gate, inplace=True)
qc.measure_all()
print(t.counts(qc))
print(qc.draw())

{'010111': 997, '101000': 1013, '000000': 1088, '111001': 1037, '000110': 1002, '010001': 1030, '101110': 1061, '111111': 964}
        ┌───┐           ░ ┌─┐               
   q_0: ┤ H ├──■────────░─┤M├───────────────
        ├───┤  │        ░ └╥┘┌─┐            
   q_1: ┤ H ├──┼────■───░──╫─┤M├────────────
        └───┘┌─┴─┐  │   ░  ║ └╥┘┌─┐         
   q_2: ─────┤ X ├──┼───░──╫──╫─┤M├─────────
        ┌───┐└───┘  │   ░  ║  ║ └╥┘┌─┐      
   q_3: ┤ H ├──■────┼───░──╫──╫──╫─┤M├──────
        └───┘┌─┴─┐  │   ░  ║  ║  ║ └╥┘┌─┐   
   q_4: ─────┤ X ├──┼───░──╫──╫──╫──╫─┤M├───
             └───┘┌─┴─┐ ░  ║  ║  ║  ║ └╥┘┌─┐
   q_5: ──────────┤ X ├─░──╫──╫──╫──╫──╫─┤M├
                  └───┘ ░  ║  ║  ║  ║  ║ └╥┘
meas: 6/═══════════════════╩══╩══╩══╩══╩══╩═
                           0  1  2  3  4  5 


In [6]:
# build T|0> = |talks> and A|0> = |answers> operators (T = A)

qr = qk.QuantumRegister(2, 'q')
qc = qk.QuantumCircuit(qr)

qc.h(0)
qc.cnot(0, 1)

print(qc.draw())

A_gate = deepcopy(qc)
T_gate = deepcopy(qc)

     ┌───┐     
q_0: ┤ H ├──■──
     └───┘┌─┴─┐
q_1: ─────┤ X ├
          └───┘


In [7]:
# build Tdag_gate

qr = qk.QuantumRegister(2, 'q')
qc = qk.QuantumCircuit(qr)

qc.cnot(0, 1)
qc.h(0)

print(qc.draw())

Adag_gate = deepcopy(qc)
Tdag_gate = deepcopy(qc)

          ┌───┐
q_0: ──■──┤ H ├
     ┌─┴─┐└───┘
q_1: ┤ X ├─────
     └───┘     


In [8]:
# show that |talks> and |answers> are correct
qr = qk.QuantumRegister(2, 'q')
qc = qk.QuantumCircuit(qr)

qc.compose(A_gate, inplace=True)
qc.measure_all()
print(t.counts(qc))
print(qc.draw())

{'00': 4039, '11': 4153}
        ┌───┐      ░ ┌─┐   
   q_0: ┤ H ├──■───░─┤M├───
        └───┘┌─┴─┐ ░ └╥┘┌─┐
   q_1: ─────┤ X ├─░──╫─┤M├
             └───┘ ░  ║ └╥┘
meas: 2/══════════════╩══╩═
                      0  1 


In [9]:
# build calA gate
wrap=True

qr = qk.QuantumRegister(10, 'q')
qc = qk.QuantumCircuit(qr)

# state initialization
qc.compose(W_gate, qr[:6], inplace=True, wrap=wrap)
qc.compose(T_gate, qr[6:8], inplace=True, wrap=wrap)
qc.compose(A_gate, qr[8:10], inplace=True, wrap=wrap)

qc.barrier()

# first contraction
qc.cnot(5, 6)
qc.h(5)

qc.barrier()

# second contraction
qc.cnot(4, 7)
qc.h(4)

qc.barrier()

# third contraction
qc.cnot(1, 8)
qc.h(1)

qc.barrier()

# fourth contraction
qc.cnot(0, 9)
qc.h(0)

print(qc.draw())

calA_gate = deepcopy(qc)

      ┌────────────┐  ░            ░            ░            ░      ┌───┐
q_0: ─┤0           ├──░────────────░────────────░────────────░───■──┤ H ├
      │            │  ░            ░            ░      ┌───┐ ░   │  └───┘
q_1: ─┤1           ├──░────────────░────────────░───■──┤ H ├─░───┼───────
      │            │  ░            ░            ░   │  └───┘ ░   │       
q_2: ─┤2           ├──░────────────░────────────░───┼────────░───┼───────
      │  circuit-3 │  ░            ░            ░   │        ░   │       
q_3: ─┤3           ├──░────────────░────────────░───┼────────░───┼───────
      │            │  ░            ░      ┌───┐ ░   │        ░   │       
q_4: ─┤4           ├──░────────────░───■──┤ H ├─░───┼────────░───┼───────
      │            │  ░      ┌───┐ ░   │  └───┘ ░   │        ░   │       
q_5: ─┤5           ├──░───■──┤ H ├─░───┼────────░───┼────────░───┼───────
     ┌┴────────────┴┐ ░ ┌─┴─┐└───┘ ░   │        ░   │        ░   │       
q_6: ┤0             ├─░─┤ X ├──────░──

In [10]:
# build calAdag gate
wrap=True

qr = qk.QuantumRegister(10, 'q')
qc = qk.QuantumCircuit(qr)

# fourth contraction
qc.cnot(0, 9)
qc.h(0)

qc.barrier()

# third contraction
qc.cnot(1, 8)
qc.h(1)

qc.barrier()

# second contraction
qc.cnot(4, 7)
qc.h(4)

qc.barrier()

# first contraction
qc.cnot(5, 6)
qc.h(5)

qc.barrier()

qc.compose(Wdag_gate, qr[:6], inplace=True, wrap=wrap)
qc.compose(Tdag_gate, qr[6:8], inplace=True, wrap=wrap)
qc.compose(Adag_gate, qr[8:10], inplace=True, wrap=wrap)

print(qc.draw())

calAdag_gate = deepcopy(qc)

          ┌───┐ ░            ░            ░            ░  ┌────────────┐ 
q_0: ──■──┤ H ├─░────────────░────────────░────────────░──┤0           ├─
       │  └───┘ ░      ┌───┐ ░            ░            ░  │            │ 
q_1: ──┼────────░───■──┤ H ├─░────────────░────────────░──┤1           ├─
       │        ░   │  └───┘ ░            ░            ░  │            │ 
q_2: ──┼────────░───┼────────░────────────░────────────░──┤2           ├─
       │        ░   │        ░            ░            ░  │  circuit-4 │ 
q_3: ──┼────────░───┼────────░────────────░────────────░──┤3           ├─
       │        ░   │        ░      ┌───┐ ░            ░  │            │ 
q_4: ──┼────────░───┼────────░───■──┤ H ├─░────────────░──┤4           ├─
       │        ░   │        ░   │  └───┘ ░      ┌───┐ ░  │            │ 
q_5: ──┼────────░───┼────────░───┼────────░───■──┤ H ├─░──┤5           ├─
       │        ░   │        ░   │        ░ ┌─┴─┐└───┘ ░ ┌┴────────────┴┐
q_6: ──┼────────░───┼────────░───┼────

In [11]:
# build Sx gate

qr = qk.QuantumRegister(11, 'q')
qc = qk.QuantumCircuit(qr)

qc.x([i for i in range(11) if i not in [2, 3, 10]])
qc.compose(MCMT('cx', 9, 1), qr[:2] + qr[3:], inplace=True)
qc.x([i for i in range(11) if i not in [2, 3, 10]])


print(qc.draw())

Sx_gate = deepcopy(qc)

      ┌───┐     ┌───┐
 q_0: ┤ X ├──■──┤ X ├
      ├───┤  │  ├───┤
 q_1: ┤ X ├──■──┤ X ├
      └───┘  │  └───┘
 q_2: ───────┼───────
             │       
 q_3: ───────■───────
      ┌───┐  │  ┌───┐
 q_4: ┤ X ├──■──┤ X ├
      ├───┤  │  ├───┤
 q_5: ┤ X ├──■──┤ X ├
      ├───┤  │  ├───┤
 q_6: ┤ X ├──■──┤ X ├
      ├───┤  │  ├───┤
 q_7: ┤ X ├──■──┤ X ├
      ├───┤  │  ├───┤
 q_8: ┤ X ├──■──┤ X ├
      ├───┤  │  ├───┤
 q_9: ┤ X ├──■──┤ X ├
      └───┘┌─┴─┐└───┘
q_10: ─────┤ X ├─────
           └───┘     


In [12]:
# build S0 gate

qr = qk.QuantumRegister(10, 'q')
qc = qk.QuantumCircuit(qr)

affected = qr[:2] + qr[3:]

qc.x(affected)
qc.h(-1)
qc.compose(MCMT('cx', 8, 1), affected, inplace=True)
qc.h(-1)
qc.x(affected)

print(qc.draw())

S0_gate = deepcopy(qc)

     ┌───┐          ┌───┐     
q_0: ┤ X ├───────■──┤ X ├─────
     ├───┤       │  ├───┤     
q_1: ┤ X ├───────■──┤ X ├─────
     └───┘       │  └───┘     
q_2: ────────────┼────────────
     ┌───┐       │  ┌───┐     
q_3: ┤ X ├───────■──┤ X ├─────
     ├───┤       │  ├───┤     
q_4: ┤ X ├───────■──┤ X ├─────
     ├───┤       │  ├───┤     
q_5: ┤ X ├───────■──┤ X ├─────
     ├───┤       │  ├───┤     
q_6: ┤ X ├───────■──┤ X ├─────
     ├───┤       │  ├───┤     
q_7: ┤ X ├───────■──┤ X ├─────
     ├───┤       │  ├───┤     
q_8: ┤ X ├───────■──┤ X ├─────
     ├───┤┌───┐┌─┴─┐├───┤┌───┐
q_9: ┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├
     └───┘└───┘└───┘└───┘└───┘


In [13]:
# build Q operator 

qr = qk.QuantumRegister(11, 'q')
qc = qk.QuantumCircuit(qr)

qc.compose(Sx_gate, qr, inplace=True)
qc.barrier()
qc.compose(calAdag_gate, qr[:-1], inplace=True)
qc.barrier()
qc.compose(S0_gate, qr[:-1], inplace=True)
qc.barrier()
qc.compose(calA_gate, qr[:-1], inplace=True)
qc.barrier()

print(qc.draw())

Q_gate = deepcopy(qc)

      ┌───┐     ┌───┐ ░      ┌───┐ ░            ░            ░            ░ »
 q_0: ┤ X ├──■──┤ X ├─░───■──┤ H ├─░────────────░────────────░────────────░─»
      ├───┤  │  ├───┤ ░   │  └───┘ ░      ┌───┐ ░            ░            ░ »
 q_1: ┤ X ├──■──┤ X ├─░───┼────────░───■──┤ H ├─░────────────░────────────░─»
      └───┘  │  └───┘ ░   │        ░   │  └───┘ ░            ░            ░ »
 q_2: ───────┼────────░───┼────────░───┼────────░────────────░────────────░─»
             │        ░   │        ░   │        ░            ░            ░ »
 q_3: ───────■────────░───┼────────░───┼────────░────────────░────────────░─»
      ┌───┐  │  ┌───┐ ░   │        ░   │        ░      ┌───┐ ░            ░ »
 q_4: ┤ X ├──■──┤ X ├─░───┼────────░───┼────────░───■──┤ H ├─░────────────░─»
      ├───┤  │  ├───┤ ░   │        ░   │        ░   │  └───┘ ░      ┌───┐ ░ »
 q_5: ┤ X ├──■──┤ X ├─░───┼────────░───┼────────░───┼────────░───■──┤ H ├─░─»
      ├───┤  │  ├───┤ ░   │        ░   │        ░   │        ░ ┌

In [14]:
# test

max_iter = 100

qr = qk.QuantumRegister(11, 'q')
cr = qk.ClassicalRegister(10, 'c')
qc = qk.QuantumCircuit(qr, cr)

qc.compose(calA_gate, qr[:-1], inplace=True)
qc.x(qr[-1])
qc.h(qr[-1])

qc.barrier()

# print(qc.draw())

for i in range(1, max_iter+1):
    qc0 = deepcopy(qc)
    for j in range(i):
        qc0.compose(Q_gate, qr, inplace=True)

    qc0.measure(qr[:-1], cr)

#     print(qc0.draw())
    # i want the state |001>
    shots = 2**16
    print('iter ', i)
    counts = t.counts(qc0, shots=shots)
    print(counts.get('0011000000', 0)/shots)
#     print(i, 'iter', counts)
#     histo(counts)

iter  1
0.0032501220703125
iter  2
0.006317138671875
iter  3
0.00030517578125
iter  4
0.000213623046875
iter  5
0.0178680419921875
iter  6
0.0086669921875
iter  7
0.0046844482421875
iter  8
0.0
iter  9
0.00439453125
iter  10
0.0028076171875
iter  11
0.0003662109375
iter  12
0.007659912109375
iter  13
0.0024871826171875
iter  14
0.016021728515625
iter  15
0.0011444091796875
iter  16
0.0027618408203125
iter  17
0.0080108642578125
iter  18
0.010955810546875
iter  19
0.0005645751953125
iter  20
0.003814697265625
iter  21
0.0
iter  22
0.003143310546875
iter  23
0.0009918212890625
iter  24
0.020355224609375
iter  25
0.0001220703125
iter  26
0.012939453125
iter  27
0.00189208984375
iter  28
0.008758544921875
iter  29
9.1552734375e-05
iter  30
0.0022430419921875
iter  31
0.00042724609375
iter  32
0.0124053955078125
iter  33
0.0070037841796875
iter  34
0.0001068115234375
iter  35
0.0002899169921875
iter  36
0.0236663818359375
iter  37
0.0046539306640625
iter  38
0.0019683837890625
iter  39
7.62